In [1]:
print('Ritu')

Ritu


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Lasso,Ridge,LinearRegression
import warnings
from sklearn.metrics import classification_report
# from sklearn.pipeline import Pipeline


In [3]:
df = pd.read_csv('Melbourne_housing_FULL.csv')
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34857 entries, 0 to 34856
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         34857 non-null  object 
 1   Address        34857 non-null  object 
 2   Rooms          34857 non-null  int64  
 3   Type           34857 non-null  object 
 4   Price          27247 non-null  float64
 5   Method         34857 non-null  object 
 6   SellerG        34857 non-null  object 
 7   Date           34857 non-null  object 
 8   Distance       34856 non-null  float64
 9   Postcode       34856 non-null  float64
 10  Bedroom2       26640 non-null  float64
 11  Bathroom       26631 non-null  float64
 12  Car            26129 non-null  float64
 13  Landsize       23047 non-null  float64
 14  BuildingArea   13742 non-null  float64
 15  YearBuilt      15551 non-null  float64
 16  CouncilArea    34854 non-null  object 
 17  Lattitude      26881 non-null  float64
 18  Longti

In [5]:
df.shape

(34857, 21)

In [6]:
df.isnull().sum()

,0
Suburb,0
Address,0
Rooms,0
Type,0
Price,7610
Method,0
SellerG,0
Date,0
Distance,1
Postcode,1


In [7]:
df.dropna(subset=['Price'],inplace = True)

In [8]:
df.shape

(27247, 21)

In [9]:
null_data = df.isna().sum()/len(df)*100
null_data

,0
Suburb,0.000000
Address,0.000000
Rooms,0.000000
Type,0.000000
Price,0.000000
Method,0.000000
SellerG,0.000000
Date,0.000000
Distance,0.003670
Postcode,0.003670


In [10]:
col_les_3 = list(null_data[(null_data<=3) & (null_data>0)].index)
col_les_3

['Distance', 'Postcode', 'CouncilArea', 'Regionname', 'Propertycount']

In [11]:
df.dropna(subset = col_les_3,inplace = True)

In [12]:
df.isna().sum()/len(df)*100

,0
Suburb,0.000000
Address,0.000000
Rooms,0.000000
Type,0.000000
Price,0.000000
Method,0.000000
SellerG,0.000000
Date,0.000000
Distance,0.000000
Postcode,0.000000


## Creating data into two capy one with grater than 50% of null value with imputation and scoend wiht remove feature which has null value grater than 50%

In [13]:
print('mean :',df.Bedroom2.mean())
print('mode :',df.Bedroom2.mode()[0])
print('median :',df.Bedroom2.median())
df.Bedroom2.value_counts()

mean : 3.0462366625012014
mode : 3.0
median : 3.0


,count
Bedroom2,
3.0,9416
4.0,4766
2.0,4696
5.0,983
1.0,781
6.0,110
7.0,19
0.0,16
8.0,9


In [14]:
print('mean :',df.Bathroom.mean())
print('mode :',df.Bathroom.mode()[0])
print('median :',df.Bathroom.median())
df.Bathroom.value_counts()

mean : 1.5916826923076923
mode : 1.0
median : 1.0


,count
Bathroom,
1.0,10527
2.0,8464
3.0,1537
4.0,176
5.0,46
0.0,34
6.0,10
8.0,3
7.0,2


In [15]:
df.Bedroom2.fillna(df.Bedroom2.mode()[0],inplace =True)

<ipython-input-15-f1e3f7b5516b>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.Bedroom2.fillna(df.Bedroom2.mode()[0],inplace =True)


In [16]:
print('mean :',df.Car.mean())
print('mode :',df.Car.mode()[0])
print('median :',df.Car.median())
df.Car.value_counts()

mean : 1.7153699260637516
mode : 2.0
median : 2.0


,count
Car,
2.0,9367
1.0,7400
0.0,1251
3.0,1234
4.0,907
5.0,113
6.0,105
7.0,19
8.0,18


In [17]:
df.Bedroom2.fillna(df.Bedroom2.mode()[0],inplace =True)

<ipython-input-17-f1e3f7b5516b>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.Bedroom2.fillna(df.Bedroom2.mode()[0],inplace =True)


In [18]:
print('mean :',df.Landsize.mean())
print('mode :',df.Landsize.mode()[0])
print('median :',df.Landsize.median())
df.Landsize.value_counts()

mean : 593.4889333778223
mode : 0.0
median : 512.0


,count
Landsize,
0.0,1942
650.0,161
697.0,97
585.0,74
534.0,70
...,...
1276.0,1
1026.0,1
1803.0,1


In [19]:
print('mean :',df.Lattitude.mean())
print('mode :',df.Lattitude.mode()[0])
print('median :',df.Lattitude.median())
df.Lattitude.value_counts()

mean : -37.80696302719954
mode : -37.8361
median : -37.80046


,count
Lattitude,
-37.83610,22
-37.79690,17
-37.84240,16
-37.79560,15
-37.84140,14
...,...
-37.89416,1
-37.88887,1
-37.93365,1


In [20]:
print('mean :',df.Longtitude.mean())
print('mode :',df.Longtitude.mode()[0])
print('median :',df.Longtitude.median())
df.Longtitude.value_counts()

mean : 144.99671101938742
mode : 144.9966
median : 145.0032


,count
Longtitude,
144.99660,17
145.01040,15
144.98500,15
144.99100,14
144.98910,13
...,...
145.05357,1
145.07929,1
145.05919,1


In [21]:
df.Bedroom2.fillna(df.Bedroom2.mode()[0],inplace =True)
df.Bathroom.fillna(df.Bathroom.mode()[0],inplace =True)
df.Car.fillna(df.Car.mode()[0],inplace =True)
df.Landsize.fillna(df.Landsize.mode()[0],inplace =True)
df.Lattitude.fillna(df.Lattitude.mode()[0],inplace =True)
df.Longtitude.fillna(df.Longtitude.mode()[0],inplace =True)

<ipython-input-21-c4ba9e556bf2>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.Bedroom2.fillna(df.Bedroom2.mode()[0],inplace =True)
<ipython-input-21-c4ba9e556bf2>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', t

In [22]:
df.isna().sum()/len(df)*100

,0
Suburb,0.000000
Address,0.000000
Rooms,0.000000
Type,0.000000
Price,0.000000
Method,0.000000
SellerG,0.000000
Date,0.000000
Distance,0.000000
Postcode,0.000000


### with 50%

In [23]:
print('mean :',df.BuildingArea.mean())
print('mode :',df.BuildingArea.mode()[0])
print('median :',df.BuildingArea.median())
df.BuildingArea.value_counts()

mean : 156.83458555743243
mode : 120.0
median : 133.0


,count
BuildingArea,
120.00,152
110.00,128
115.00,126
100.00,123
130.00,121
...,...
167.13,1
443.00,1
104.63,1


In [24]:
print('mean :',df.YearBuilt.mean())
print('mode :',df.YearBuilt.mode()[0])
print('median :',df.YearBuilt.median())
df.YearBuilt.value_counts()

mean : 1966.6091525984773
mode : 1970.0
median : 1970.0


,count
YearBuilt,
1970.0,1194
1960.0,1013
1950.0,844
1980.0,575
1900.0,452
...,...
1932.0,1
1854.0,1
1830.0,1


In [25]:
YearBuilt = df.YearBuilt.fillna(df.YearBuilt.mode()[0])
BuildingArea = df.BuildingArea.fillna(df.BuildingArea.mean())
df1 = df.drop(['YearBuilt','BuildingArea'],axis = 1)
df1['BuildingArea'] = BuildingArea
df1['YearBuilt'] = YearBuilt
print(df.shape,df1.shape)

(27244, 21) (27244, 21)


In [26]:
df.isna().sum()/len(df)*100

,0
Suburb,0.000000
Address,0.000000
Rooms,0.000000
Type,0.000000
Price,0.000000
Method,0.000000
SellerG,0.000000
Date,0.000000
Distance,0.000000
Postcode,0.000000


In [27]:
df1.isna().sum()/len(df)*100

,0
Suburb,0.0
Address,0.0
Rooms,0.0
Type,0.0
Price,0.0
Method,0.0
SellerG,0.0
Date,0.0
Distance,0.0
Postcode,0.0


### Without 50%

In [28]:
df2 = df.drop(['BuildingArea','YearBuilt'],axis = 1)
df.shape,df2.shape


((27244, 21), (27244, 19))

In [29]:
df2.isna().sum()/len(df)*100

,0
Suburb,0.0
Address,0.0
Rooms,0.0
Type,0.0
Price,0.0
Method,0.0
SellerG,0.0
Date,0.0
Distance,0.0
Postcode,0.0


In [30]:
df2.Date = pd.to_datetime(df2.Date,format = '%d/%m/%Y')

In [31]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27244 entries, 1 to 34856
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Suburb         27244 non-null  object        
 1   Address        27244 non-null  object        
 2   Rooms          27244 non-null  int64         
 3   Type           27244 non-null  object        
 4   Price          27244 non-null  float64       
 5   Method         27244 non-null  object        
 6   SellerG        27244 non-null  object        
 7   Date           27244 non-null  datetime64[ns]
 8   Distance       27244 non-null  float64       
 9   Postcode       27244 non-null  float64       
 10  Bedroom2       27244 non-null  float64       
 11  Bathroom       27244 non-null  float64       
 12  Car            27244 non-null  float64       
 13  Landsize       27244 non-null  float64       
 14  CouncilArea    27244 non-null  object        
 15  Lattitude      27244 non

In [32]:
df2['year'] = df2.Date.dt.year
df2['month'] = df2.Date.dt.month
df2['day'] = df2.Date.dt.day
df2.drop('Date',axis = 1,inplace = True)
df2.shape

(27244, 21)

In [33]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27244 entries, 1 to 34856
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         27244 non-null  object 
 1   Address        27244 non-null  object 
 2   Rooms          27244 non-null  int64  
 3   Type           27244 non-null  object 
 4   Price          27244 non-null  float64
 5   Method         27244 non-null  object 
 6   SellerG        27244 non-null  object 
 7   Distance       27244 non-null  float64
 8   Postcode       27244 non-null  float64
 9   Bedroom2       27244 non-null  float64
 10  Bathroom       27244 non-null  float64
 11  Car            27244 non-null  float64
 12  Landsize       27244 non-null  float64
 13  CouncilArea    27244 non-null  object 
 14  Lattitude      27244 non-null  float64
 15  Longtitude     27244 non-null  float64
 16  Regionname     27244 non-null  object 
 17  Propertycount  27244 non-null  float64
 18  year       

In [34]:
print(len(df2.Suburb.unique()))
print(len(df2.Address.unique()))
print(len(df2.Type.unique()))
print(len(df2.Method.unique()))
print(len(df2.SellerG.unique()))
print(len(df2.CouncilArea.unique()))
print(len(df2.Regionname.unique()))

344
26748
3
5
349
33
8


In [35]:
df2.shape

(27244, 21)

In [36]:
df2.drop('Address',axis = 1,inplace = True)

In [37]:
df2.shape

(27244, 20)

In [38]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27244 entries, 1 to 34856
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         27244 non-null  object 
 1   Rooms          27244 non-null  int64  
 2   Type           27244 non-null  object 
 3   Price          27244 non-null  float64
 4   Method         27244 non-null  object 
 5   SellerG        27244 non-null  object 
 6   Distance       27244 non-null  float64
 7   Postcode       27244 non-null  float64
 8   Bedroom2       27244 non-null  float64
 9   Bathroom       27244 non-null  float64
 10  Car            27244 non-null  float64
 11  Landsize       27244 non-null  float64
 12  CouncilArea    27244 non-null  object 
 13  Lattitude      27244 non-null  float64
 14  Longtitude     27244 non-null  float64
 15  Regionname     27244 non-null  object 
 16  Propertycount  27244 non-null  float64
 17  year           27244 non-null  int32  
 18  month      

In [39]:
colt = ColumnTransformer([('ohe',OneHotEncoder(drop = 'first'),['Suburb','Type','Method','SellerG','CouncilArea','Regionname'])],
                        remainder = 'passthrough')

In [40]:
X = df2.drop('Price',axis = 1)
y=df2.Price

In [41]:
X = colt.fit_transform(X)

In [42]:
X.shape

(27244, 749)

In [43]:
X1 = X.toarray()

In [44]:
X2 = X.toarray()

In [45]:
X1 = pd.DataFrame(X1)
X1['YearBuilt'] = YearBuilt
X1['BuildingArea'] = BuildingArea
X1.columns = X1.columns.astype(str)

In [46]:
X1['YearBuilt'].fillna(YearBuilt.mode()[0],inplace = True)
X1['BuildingArea'].fillna(BuildingArea.mean(),inplace  = True)
X1.shape

<ipython-input-46-a0219e02de69>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X1['YearBuilt'].fillna(YearBuilt.mode()[0],inplace = True)
<ipython-input-46-a0219e02de69>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)'

(27244, 751)

In [47]:
X1.isna().sum()

,0
0,0
1,0
2,0
3,0
4,0
...,...
746,0
747,0
748,0
YearBuilt,0


In [48]:
X_train1,X_test1,y_train1,y_test1 = train_test_split(X1,y,test_size = 0.23,random_state = 42)
X_train2,X_test2,y_train2,y_test2 = train_test_split(X2,y,test_size = 0.23,random_state = 42)

In [49]:
lr1 = LinearRegression()
lr1.fit(X_train1,y_train1)
print('lr1 score: ',lr1.score(X_test1,y_test1))
lr2 = LinearRegression()
lr2.fit(X_train2,y_train2)
print('lr2 score: ',lr2.score(X_test2,y_test2))

lr1 score:  0.6862906169442944
lr2 score:  0.6863132559063104


In [50]:
print('lr1 train score: ',lr1.score(X_train1,y_train1))
print('lr2 train score: ',lr2.score(X_train2,y_train2))


lr1 train score:  0.6802581675179666
lr2 train score:  0.6802465118666607


In [64]:
l = Lasso(alpha = 50)
# l = Lasso(alpha = 1,max_iter= 100)
l.fit(X_train1,y_train1)
print('l train score: ',l.score(X_train1,y_train1))
print('l test score: ',l.score(X_test1,y_test1))
r = Ridge(alpha = 50)
r.fit(X_train2,y_train2)
print('r train score: ',r.score(X_train2,y_train2))
print('r test score: ',r.score(X_test2,y_test2))

l train score:  0.6759216950545596
l test score:  0.6908965223870318
r train score:  0.6633479676469166
r test score:  0.6822218489104136
